In [ ]:
import matplotlib.pyplot as plt
plt.plot()
plt.close()

In [ ]:
import torch
from torch import nn, optim
from torchvision import transforms, models
from tqdm import tqdm
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.distributions as dist
import os
from density_estimation import *
from nonlinear_vae import *
from custom_data_loading import *
from train_vae import *
from distance_correlation import *
from spearman_correlation import *

In [ ]:
tensor_path='/kaggle/input/0618-mixed-stress-path'
#tensor_path='/kaggle/input/scaled-stress-with-channel-regarding2dataset/Set3'
file_list=[]
for dirname, _, filenames in os.walk(tensor_path):
    for filename in filenames:
        file_list.append(os.path.join(dirname, filename))
print('The data set size:',len(file_list))
#Remove the initial state
import re
for i in file_list:
#j=0
    pt_name=i.split('\\')[-1]
    if re.findall(r'\d+', pt_name)[-1]=='0':
        file_list.remove(i)
print('The data set size:',len(file_list))

train_list,test_list=list_splitting(file_list,proportion=0.2,shuffle=True)

In [ ]:
tensor_path='/kaggle/input/0618-mixed-stress-path'
#tensor_path='/kaggle/input/scaled-stress-with-channel-regarding2dataset/Set3'
file_list=[]
for dirname, _, filenames in os.walk(tensor_path):
    for filename in filenames:
        file_list.append(os.path.join(dirname, filename))
print('The data set size:',len(file_list))
#Remove the initial state
import re
for i in file_list:
#j=0
    pt_name=i.split('\\')[-1]
    if re.findall(r'\d+', pt_name)[-1]=='0':
        file_list.remove(i)
print('The data set size:',len(file_list))

train_list,test_list=list_splitting(file_list,proportion=0.2,shuffle=True)

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"
param={'lr':0.00001,
       'max_epoch':125,
       'batch_sz':32,
       'Type':'Covariance',
       'beta':0,
       'alpha':0,
       'l1':0,
       'l2':10,
       'l3':10,
       'by_channel':True,
       'channel_idx':0,
       'normalize':False,
       'loss_fn':elbo_loss_SC,
    }

In [ ]:
M=NonLinearVAE(fc_list=[256*25*25,1024,521,128,64,32])
M.to(device)

In [ ]:
trained_M=VAE_list_trainer(M,train_list,test_list,param)

In [ ]:
plot_loss_reg_VAE(trained_M,mode='Training_only')


In [ ]:
x=load_a_batch_tensor(train_list[:32],by_channel=True).to(device)
_,z,x_hat=M.forward(x)
i=1
plt.matshow(x[i,0].cpu().detach().numpy())

In [ ]:
plt.matshow(x_hat[i,0].cpu().detach().numpy())

In [ ]:
profile = {'Trained_model':trained_M,\
            'Trained_param':param,\
           'state_dict': trained_M.state_dict(),}
torch.save(profile,'0626_SkVAE_SP_zShape32.pth')